## 1. 학습 데이터의 월 판매량 및 모든 시차 피쳐, 예측결과 (0,20)범위로 clip

## 2. hyper parameter 지정
- 'metric': 'rmse', # 평가지표 = rmse
- 'num_leaves': 255, # reaf node 수
- 'learning_rate': 0.005, # 디폴트0.1, 작을수록 예측성능이 높아질수있지만 과적합가능성도 올라감
- 'feature_fraction': 0.75, # 디폴트 1,개별 트리를 학습할 때 무작위로 선택되는 피쳐 비율
- 'bagging_fraction': 0.75, # 디폴트 1, 학습데이터 샘플링 비율
- 'bagging_freq': 5, # 디폴트 0, 몇번 iteration할때마다 다시 bagging을 수행할 지
- 'force_col_wise': True, # 디폴트 false, 컬럼수가 많을떄 메모리 비용을 줄일 수 있음
- 'random_state': 42 # seed 고정
- 'num_boost_round' : 1500 # 디폴트 100, 반복수행하는 횟수 
- 'early_stopping_rounds' : 150 # 디폴트 0, 검증결과가 150번동안 나아지지 않으면 멈춤
- 'categorical_feature' : ['shop_id', 'city_id', 'cat_id', 'main_cat', 'sub_cat', 'year','month'] # 범주형 피쳐 지정
- 'verbose_eval'=100 # 몇번째마다 평가지표 출력할지

## 3. 사용 데이터
- data_3 : 이상값 단순 제거(보고서 내 전처리 항목 참고)

## 4. submission 여부 : YES
- Score: 0.88958, 1528/13598 ~ 상위 11.2%

In [1]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import time
from itertools import product 
import lightgbm

In [3]:
data = pd.read_pickle('./data/data_3.pkl')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8029111 entries, 827196 to 8856306
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   month_id                    int8   
 1   shop_id                     int8   
 2   item_id                     int16  
 3   cnt_month                   float32
 4   city_id                     int8   
 5   cat_id                      int8   
 6   main_cat                    int8   
 7   sub_cat                     int8   
 8   year                        int16  
 9   month                       int8   
 10  sales_month                 int8   
 11  cnt_month_1                 float32
 12  cnt_month_2                 float32
 13  cnt_month_3                 float32
 14  item_id_mean_cnt_1          float32
 15  item_id_mean_cnt_2          float32
 16  item_id_mean_cnt_3          float32
 17  sub_cat_mean_cnt_1          float32
 18  sub_cat_mean_cnt_2          float32
 19  sub_cat_mean_cnt

In [4]:
data.isnull().sum()

month_id                           0
shop_id                            0
item_id                            0
cnt_month                     214200
city_id                            0
cat_id                             0
main_cat                           0
sub_cat                            0
year                               0
month                              0
sales_month                        0
cnt_month_1                        0
cnt_month_2                        0
cnt_month_3                        0
item_id_mean_cnt_1                 0
item_id_mean_cnt_2                 0
item_id_mean_cnt_3                 0
sub_cat_mean_cnt_1                 0
sub_cat_mean_cnt_2                 0
sub_cat_mean_cnt_3                 0
price_mean_mean_cnt_1              0
price_mean_mean_cnt_2              0
price_mean_mean_cnt_3              0
item_id/city_id_mean_cnt_1         0
item_id/city_id_mean_cnt_2         0
item_id/city_id_mean_cnt_3         0
shop_id/cat_id_mean_cnt_1          0
s

In [5]:
data[['cnt_month','cnt_month_1','cnt_month_2','cnt_month_3']]=data[['cnt_month','cnt_month_1','cnt_month_2','cnt_month_3']].clip(0,20)

In [6]:
# train/validation/test data split
X_train = data[data['month_id'] < 33].drop(['cnt_month'], axis=1)
y_train = data[data['month_id'] < 33]['cnt_month']

X_valid = data[data['month_id'] == 33].drop(['cnt_month'], axis=1)
y_valid = data[data['month_id'] == 33]['cnt_month']

X_test = data[data['month_id'] == 34].drop(['cnt_month'], axis=1)

In [8]:
# hyper parameter 지정
params = {'metric': 'rmse', # 평가지표 = rmse
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 42}

cat = ['shop_id', 'city_id', 'cat_id', 'main_cat', 'sub_cat', 'year','month']

# 데이터 셋 지정
train = lightgbm.Dataset(X_train, y_train)
valid = lightgbm.Dataset(X_valid, y_valid)
 
# 모델 훈련
start = time.time()
lgb_model = lightgbm.train(params=params,
                      train_set=train,
                      num_boost_round=1500,
                      valid_sets=(train, valid),
                      early_stopping_rounds=150,
                      categorical_feature=cat,
                      verbose_eval=100)
end = time.time()
print(f'소요시간(s) : {end-start}')

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_id', 'city_id', 'main_cat', 'month', 'shop_id', 'sub_cat', 'year']
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Info] Total Bins 4314
[LightGBM] [Info] Number of data points in the train set: 7593193, number of used features: 28
[LightGBM] [Info] Start training from score 0.307227
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.02085	valid_1's rmse: 0.913752
[200]	training's rmse: 0.91745	valid_1's rmse: 0.843846
[300]	training's rmse: 0.865265	valid_1's rmse: 0.815466
[400]	training's rmse: 0.836517	valid_1's rmse: 0.805331
[500]	training's rmse: 0.818505	valid_1's rmse: 0.801538
[600]	training's rmse: 0.805912	valid_1's rmse: 0.800214
[700]	training's rmse: 0.794929	valid_1's rmse: 0.799744
[800]	training's rmse: 0.78613	valid_1's rmse: 0.799744
[900]	training's rmse: 0.778629	valid_1's rmse: 0.799763
[1000]	training's rmse: 0.772618	valid_1's rmse: 0.799652
[1100]	training's rmse: 0.767667	valid_1's rmse: 0.799726
Early stopping, best iteration is:
[982]	training's rmse: 0.773593	valid_1's rmse: 0.799417
소요시간(s) : 706.5009162425995


In [10]:
# 예측 수행
y_pred = lgb_model.predict(X_test).clip(0, 20) # 실제 타겟값이 클립되어있으므로 동일하게 클립

In [11]:
# 제출 파일 생성
df_test = pd.read_csv('e:/Git_public_dodo_Riley/kaggle/Future_Sales/data/test.csv')
submission = pd.DataFrame({"ID": df_test.index, "item_cnt_month": y_pred})
submission.to_csv('./submission file/lgbm_submission_7.csv', index=False)